In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
def corr2d(X,K):
    """计算二维互相关运算"""
    h,w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1),X.shape[1] - w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            # 模拟卷积操作
            Y[i,j] = (X[i:i + h,j : j + w] * K).sum()

    return Y

In [2]:
# 实现二维卷积层
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()

        # 随机初始化 卷积核权重
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    # 将输入x和卷积核做互相关运算
    def forward(self,x):
        return corr2d(x,self.weight) + self.bias

In [4]:
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [7]:
#  学习卷积核  
X = torch.ones((6,8))
K = torch.tensor([[1.0,-1.0]])

Y = corr2d(X,K)

# 创建一个卷积层 4d
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)


# 创建输入和输出形状
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))

lr = 3e-2  # 学习率


for i in range(10):
    # 预测
    Y_hat = conv2d(X)

    # 计算均方差
    l = (Y_hat - Y) ** 2

    conv2d.zero_grad()  # 梯度清0

    l.sum().backward() # 误差反向传播

    # 迭代卷积核   更新梯度
    conv2d.weight.data[:] -= lr * conv2d.weight.grad

    if (i + 1) % 2 == 0:
        print(f'epoch{i+1},loss {l.sum():.3f}')


epoch2,loss 96.154
epoch4,loss 25614.863
epoch6,loss 6823660.500
epoch8,loss 1817785728.000
epoch10,loss 484248322048.000


In [8]:
conv2d.weight.data.reshape((1, 2))

tensor([[216900.8750, 216900.4062]])